In [1]:
from datasets import load_dataset
import soundfile
import librosa
import datasets

In [2]:
from transformers import Wav2Vec2BertForCTC
model = Wav2Vec2BertForCTC.from_pretrained("facebook/w2v-bert-2.0")
model.save_model("/teamspace/studios/this_studio/models/w2v-bert-2.0")

ValueError: You are trying to instantiate <class 'transformers.models.wav2vec2_bert.modeling_wav2vec2_bert.Wav2Vec2BertForCTC'> with a configuration that does not define the vocabulary size of the language model head. Please instantiate the model as follows: `Wav2Vec2BertForCTC.from_pretrained(..., vocab_size=vocab_size)`. or define `vocab_size` of your model's configuration.

In [27]:
from datasets import dataset_dict

In [20]:
fleurs = load_dataset("google/fleurs", "he_il", split=None)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [29]:
type(fleurs) == 

True

In [ ]:
kan = load_dataset("imvladikon/hebrew_speech_kan", split="validation")

In [78]:
campus= load_dataset("imvladikon/hebrew_speech_campus")

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
coursera= load_dataset("imvladikon/hebrew_speech_coursera")

In [48]:
def standardize_datasets(datasets):
    for dataset in datasets.keys():
        print(dataset)
        data = datasets[dataset]
        data= data.remove_columns([col for col in data.features if col not in ["sentence", "transcription", "audio"]])
        try:
            data= data.rename_column("sentence", "transcription")
        except:
            pass
        data= data.map(lambda x: {'dataset': dataset})
        datasets[dataset] = data
    return datasets

In [49]:
datasets = {'kan': kan, 'fleurs': fleurs}

In [50]:
standardized_datasets = standardize_datasets(datasets)

kan
fleurs


Map:   0%|          | 0/792 [00:00<?, ? examples/s]

In [51]:
standardized_datasets

{'kan': Dataset({
     features: ['audio', 'transcription', 'dataset'],
     num_rows: 2000
 }),
 'fleurs': Dataset({
     features: ['audio', 'transcription', 'dataset'],
     num_rows: 792
 })}

In [54]:
#concatenate the datasets
from datasets import concatenate_datasets
kan_fleurs = concatenate_datasets(datasets.values())

In [55]:
kan_fleurs

Dataset({
    features: ['audio', 'transcription', 'dataset'],
    num_rows: 2792
})

In [62]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    return batch

kan_fleurs= kan_fleurs.map(remove_special_characters)
    

In [57]:
#display an example
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [77]:
import random
import pandas as pd
import string
from IPython.display import HTML, display

def show_random_english_elements_simple(dataset, text_field):
    # num_examples =len(dataset)
    # assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    
    def contains_english_letters(text):
        english_letters = set(string.ascii_lowercase)
        return any(char in english_letters for char in text.lower())
    def contains_digits(text):
        digits = set(string.digits)  # Set of all digit characters
        return any(char in digits for char in text)
    english_rows = [row for row in dataset if contains_english_letters(row[text_field]) or contains_digits(row[text_field])]
    print(len(english_rows))

show_random_english_elements_simple(kan_fleurs, 'transcription')

189


In [63]:
show_random_elements(kan_fleurs.remove_columns(['audio']), num_examples=10)

,transcription,dataset
0,זה מצב הפוך מאשר בעבר,kan
1,מעבר לאירוע של יום רביעי קרפנדו התחרתה בשני מרוצים אינדיבידואלים באליפות,fleurs
2,וגם תיקחו את הגיפה שנשארת אחרי ששוטפים כלים,kan
3,כי היא רצתה ילדים איתך דווקא איתך,kan
4,האם אתם מכוונים לאנשים שהם ספורטיביים לאנשים שאוהבים טיולים,kan
5,הפירמידה הגדולה שנבנתה על ידי המצרים במאה השלישית לפני הספירה היא אחת ממבני פירמידה גדולים ורבים שנבנו כדי לחלוק כבוד לפרעונים מתים,fleurs
6,אוטובוס 403 של סקוטורב נוסע באופן סדיר לסינטרה ועוצר בקאבו דה רוקה,fleurs
7,25 גיליונות דנלאפ שעדיין ידוע שקיימים הם העותקים העתיקים ביותר של המסמך ששרדו העותק המקורי שנכתב ביד לא שרד,fleurs
8,כבר בתחילת ספטמבר כדי שיום כיפור יסתיים מוקדם יותר,kan
9,גם בבית וגם כאימא וגם כאישה לבעלי,kan


In [68]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab = kan_fleurs.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=kan_fleurs.column_names)
vocab_list = list(set(vocab["vocab"][0])) #insert test here as well
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


Map:   0%|          | 0/2792 [00:00<?, ? examples/s]

{' ': 0,
 '$': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '[': 13,
 ']': 14,
 'a': 15,
 'b': 16,
 'c': 17,
 'd': 18,
 'e': 19,
 'f': 20,
 'g': 21,
 'h': 22,
 'i': 23,
 'j': 24,
 'k': 25,
 'l': 26,
 'm': 27,
 'n': 28,
 'o': 29,
 'p': 30,
 'r': 31,
 's': 32,
 't': 33,
 'u': 34,
 'v': 35,
 'w': 36,
 'x': 37,
 'z': 38,
 'è': 39,
 '־': 40,
 'א': 41,
 'ב': 42,
 'ג': 43,
 'ד': 44,
 'ה': 45,
 'ו': 46,
 'ז': 47,
 'ח': 48,
 'ט': 49,
 'י': 50,
 'ך': 51,
 'כ': 52,
 'ל': 53,
 'ם': 54,
 'מ': 55,
 'ן': 56,
 'נ': 57,
 'ס': 58,
 'ע': 59,
 'ף': 60,
 'פ': 61,
 'ץ': 62,
 'צ': 63,
 'ק': 64,
 'ר': 65,
 'ש': 66,
 'ת': 67,
 '–': 68}

In [69]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

71